In [10]:
import json
import pandas as pd
from rouge_score import rouge_scorer
from collections import defaultdict

train_dir = "../training_data/"
train_dir_gen = "../training_data_generated/"
task = "penguins_in_a_table"
df = pd.read_json("../training_data/penguins_in_a_table.json").to_dict()
df_aug = pd.read_json("../training_data_generated/penguins_in_a_table.json").to_dict()

In [14]:
with open(f"{train_dir}/{task}.json", "r") as f:
  ds_init = json.load(f)
f.close()
with open(f"{train_dir_gen}/{task}.json", "r") as f:
  existing_generated_questions = json.load(f)
f.close()

In [13]:
# ds_init = ds_init.extend(existing_generated_questions)

In [26]:
ds_inputs = [question["inputs"] for question in ds_init]
generated_inputs = [question["inputs"] for question in existing_generated_questions]

scorer = rouge_scorer.RougeScorer(
        ["rougeL"], use_stemmer=False
    )  # same as self-instruct setting
scores, excluded_qs, errs = defaultdict(list), set(), 0
out = []
for i, existing_q in enumerate(ds_inputs):
    for j, new_q in enumerate(generated_inputs):
        score = scorer.score(new_q, existing_q)["rougeL"].fmeasure
        scores[j].append((i, score))
        if score > 0.99:
            # self.dump_error(existing_q, self.error_file)
            # excluded_qs.add(j)
            out.append([i,j, existing_q, new_q])
            errs += 1

out

[]

In [22]:
errs

21

In [27]:
total_qs=10
prompt = f"Generate a series of {total_qs} diverse questions related to TASK DESCRIPTION in a JSON format with the following schema:"
prompt += """
```
{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "inputs": {
      "type": "string"
    },
    "targets": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "multiple_choice_targets": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  },
  "required": ["inputs", "targets", "multiple_choice_targets"]
}
```"""

In [29]:
print(prompt)

Generate a series of 10 diverse questions related to TASK DESCRIPTION in a JSON format with the following schema:
```
{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "inputs": {
      "type": "string"
    },
    "targets": {
      "type": "array",
      "items": {
        "type": "string"
      }
    },
    "multiple_choice_targets": {
      "type": "array",
      "items": {
        "type": "string"
      }
    }
  },
  "required": ["inputs", "targets", "multiple_choice_targets"]
}
```
